# Roundtrip example

This notebook shows how to load a string literal tree into Roundtrip, interact with the tree, and retrieve query information based on the selection on the interactive tree.

## Step 0: initialize Hatchet GraphFrame of data

First we load Hatchet, and os and sys to make path adjustments later.

In [1]:
import hatchet as ht
import os, sys

Next we define the string-literal version of our call tree. (Future: use Hatchet's `gf.to_literal()` function to convert your graph frame into a dictionary representation, like the one shown)

In [2]:
small_tree = [
{
    "name": "foo",
    "metrics": {"time (inc)": 130.0, "time": 0.0},
    "children": [
        {
            "name": "bar",
            "metrics": {"time (inc)": 20.0, "time": 5.0},
            "children": [
                {
                    "name": "baz",
                    "metrics": {"time (inc)": 5.0, "time": 5.0},
                },
                {
                    "name": "grault",
                    "metrics": {"time (inc)": 10.0, "time": 10.0},
                },
            ],
        },
        {
            "name": "qux",
            "metrics": {"time (inc)": 60.0, "time": 0.0},
            "children": [
                {
                    "name": "quux",
                    "metrics": {"time (inc)": 60.0, "time": 5.0},
                    "children": [
                        {
                            "name": "corge",
                            "metrics": {"time (inc)": 55.0, "time": 10.0},
                            "children": [
                                {
                                    "name": "bar",
                                    "metrics": {
                                        "time (inc)": 20.0,
                                        "time": 5.0,
                                    },
                                    "children": [
                                        {
                                            "name": "baz",
                                            "metrics": {
                                                "time (inc)": 5.0,
                                                "time": 5.0,
                                            },
                                        },
                                        {
                                            "name": "grault",
                                            "metrics": {
                                                "time (inc)": 10.0,
                                                "time": 10.0,
                                            },
                                        },
                                    ],
                                },
                                {
                                    "name": "grault",
                                    "metrics": {
                                        "time (inc)": 10.0,
                                        "time": 10.0,
                                    },
                                },
                                {
                                    "name": "garply",
                                    "metrics": {
                                        "time (inc)": 15.0,
                                        "time": 15.0,
                                    },
                                },
                            ],
                        }
                    ],
                }
            ],
        },
    ],
}
]

gf = ht.GraphFrame.from_literal(small_tree)

print(gf.dataframe)
print(gf.tree())

                      name  time (inc)  time
node                                        
{'name': 'foo'}        foo       130.0   0.0
{'name': 'bar'}        bar        20.0   5.0
{'name': 'baz'}        baz         5.0   5.0
{'name': 'grault'}  grault        10.0  10.0
{'name': 'qux'}        qux        60.0   0.0
{'name': 'quux'}      quux        60.0   5.0
{'name': 'corge'}    corge        55.0  10.0
{'name': 'bar'}        bar        20.0   5.0
{'name': 'baz'}        baz         5.0   5.0
{'name': 'grault'}  grault        10.0  10.0
{'name': 'garply'}  garply        15.0  15.0
{'name': 'grault'}  grault        10.0  10.0
    __          __       __         __ 
   / /_  ____ _/ /______/ /_  ___  / /_
  / __ \/ __ `/ __/ ___/ __ \/ _ \/ __/
 / / / / /_/ / /_/ /__/ / / /  __/ /_  
/_/ /_/\__,_/\__/\___/_/ /_/\___/\__/  v1.2.0

0.000 foo
├─ 5.000 bar
│  ├─ 5.000 baz
│  └─ 10.000 grault
└─ 0.000 qux
   └─ 5.000 quux
      └─ 10.000 corge
         ├─ 5.000 bar
         │  ├─ 5.000 baz
     

## Step 1: load Roundtrip

Here we load the IPython extension *Roundtrip*, the Python code that acts as the go-between for the visualization JavaScript and the Python code in the Jupyter notebook.

Find the relative path from this notebook to your installation of Hatchet. Roundtrip is located at `path_to_hatchet/hatchet/external/roundtrip/`. 

In [3]:
# This is the relative path from the nb to roundtrip/
rt_path = '../../../hatchet/external/roundtrip/'

# Add the path so that the notebook can find the Roundtrip extension
module_path = os.path.abspath(os.path.join(rt_path)) 
if module_path not in sys.path:
    sys.path.append(module_path)

In [4]:
%load_ext roundtrip

## Step 2: load the visualization

Next we load the custom visualization from `roundtripTree.js`. The magic function (`%loadVisualization`) and its parameters are listed below:

- `%loadVisualization`: Roundtrip function to generate the visualization
- `rt_path`: the path to Roundtrip in the Hatchet directory (path ends with `hatchet/external/roundtrip/`)
- `%small_tree`: the Python variable from cell 3 that holds the tree string literal 

After the cell is executed, the tree appears. Clicking on a node will cause its metadata to be displayed at the top of the visualization (by the "Colors" button). Double-clicking on a node will cause the subtree to collapse. 

To select a single node, click on it then execute the next cell (`%fetchData`) to retrieve its data. To select many nodes, click the button "Select nodes" to activate the brush (to turn off the brush, click "Select nodes" again).

In [5]:
%loadVisualization rt_path small_tree

<IPython.core.display.Javascript object>

## Step 3: retrieve our selection
To retrieve the data we have selected in the tree, we use the Roundtrip function `%fetchData`.

The parameters below are:
- `%fetchData`: the Roundtrip function to pass the selection from JavaScript to Python
- `myQuery`: the Python variable name we will use to store our selection


In [6]:
# Execute this cell first (without using myQuery yet)
%fetchData myQuery

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [7]:
# Now myQuery is loaded (after %fetchData executes)
print(myQuery, type(myQuery))

# If no nodes are clicked, the default behavior is to return all nodes

[{'name': 'foo'}] <class 'list'>


Now filter the graphframe `gf` by the produced query `myQuery`.

In [8]:
sgf = gf.filter(myQuery, squash=True)
print("Nodes returned:", len(sgf.graph))
print(sgf.tree())

Nodes returned: 1
    __          __       __         __ 
   / /_  ____ _/ /______/ /_  ___  / /_
  / __ \/ __ `/ __/ ___/ __ \/ _ \/ __/
 / / / / /_/ / /_/ /__/ / / /  __/ /_  
/_/ /_/\__,_/\__/\___/_/ /_/\___/\__/  v1.2.0

0.000 foo

Legend (Metric: time)
█ 0.00 - 0.00
█ 0.00 - 0.00
█ 0.00 - 0.00
█ 0.00 - 0.00
█ 0.00 - 0.00
█ 0.00 - 0.00

name User code    ◀  Only in left graph    ▶  Only in right graph



# Multiple roots/trees

The following example uses `multiple_trees`, a call tree representation that has 2 roots: foo and kap.

In [9]:
multiple_trees = [
            {
                "name": "foo",
                "metrics": {"time (inc)": 130.0, "time": 0.0},
                "children": [
                    {
                        "name": "bar",
                        "metrics": {"time (inc)": 20.0, "time": 5.0},
                        "children": [
                            {
                                "name": "baz",
                                "metrics": {"time (inc)": 5.0, "time": 5.0},
                            },
                            {
                                "name": "grault",
                                "metrics": {"time (inc)": 10.0, "time": 10.0},
                            },
                        ],
                    },
                    {
                        "name": "qux",
                        "metrics": {"time (inc)": 60.0, "time": 0.0},
                        "children": [
                            {
                                "name": "quux",
                                "metrics": {"time (inc)": 60.0, "time": 5.0},
                                "children": [
                                    {
                                        "name": "corge",
                                        "metrics": {"time (inc)": 55.0, "time": 10.0},
                                        "children": [
                                            {
                                                "name": "bar",
                                                "metrics": {
                                                    "time (inc)": 20.0,
                                                    "time": 5.0,
                                                },
                                                "children": [
                                                    {
                                                        "name": "baz",
                                                        "metrics": {
                                                            "time (inc)": 5.0,
                                                            "time": 5.0,
                                                        },
                                                    },
                                                    {
                                                        "name": "grault",
                                                        "metrics": {
                                                            "time (inc)": 10.0,
                                                            "time": 10.0,
                                                        },
                                                    },
                                                ],
                                            },
                                            {
                                                "name": "grault",
                                                "metrics": {
                                                    "time (inc)": 10.0,
                                                    "time": 10.0,
                                                },
                                            },
                                            {
                                                "name": "garply",
                                                "metrics": {
                                                    "time (inc)": 15.0,
                                                    "time": 15.0,
                                                },
                                            },
                                        ],
                                    }
                                ],
                            }
                        ],
                    },
                    {
                        "name": "waldo",
                        "metrics": {"time (inc)": 50.0, "time": 0.0},
                        "children": [
                            {
                                "name": "fred",
                                "metrics": {"time (inc)": 35.0, "time": 5.0},
                                "children": [
                                    {
                                        "name": "plugh",
                                        "metrics": {"time (inc)": 5.0, "time": 5.0},
                                    },
                                    {
                                        "name": "xyzzy",
                                        "metrics": {"time (inc)": 25.0, "time": 5.0},
                                        "children": [
                                            {
                                                "name": "thud",
                                                "metrics": {
                                                    "time (inc)": 25.0,
                                                    "time": 5.0,
                                                },
                                                "children": [
                                                    {
                                                        "name": "baz",
                                                        "metrics": {
                                                            "time (inc)": 5.0,
                                                            "time": 5.0,
                                                        },
                                                    },
                                                    {
                                                        "name": "garply",
                                                        "metrics": {
                                                            "time (inc)": 15.0,
                                                            "time": 15.0,
                                                        },
                                                    },
                                                ],
                                            }
                                        ],
                                    },
                                ],
                            },
                            {
                                "name": "garply",
                                "metrics": {"time (inc)": 15.0, "time": 15.0},
                            },
                        ],
                    },
                ],
            },
            {
                "name": "kap",
                "metrics": {"time (inc)": 30.0, "time": 0.0},
                "children": [
                    {
                        "name": "kow",
                        "metrics": {"time (inc)": 15.0, "time": 5.0},
                        "children": [
                            {
                                "name": "krazy",
                                "metrics": {"time (inc)": 5.0, "time": 5.0},
                            },
                            {
                                "name": "kraze",
                                "metrics": {"time (inc)": 5.0, "time": 5.0},
                            },
                        ],
                    },
                    {
                        "name": "kat",
                        "metrics": {"time (inc)": 15.0, "time": 15.0},
                    },
                ],
            }
        ]


multi_gf = ht.GraphFrame.from_literal(multiple_trees)

print(multi_gf.dataframe)
print(multi_gf.tree())

                      name  time (inc)  time
node                                        
{'name': 'foo'}        foo       130.0   0.0
{'name': 'bar'}        bar        20.0   5.0
{'name': 'baz'}        baz         5.0   5.0
{'name': 'grault'}  grault        10.0  10.0
{'name': 'qux'}        qux        60.0   0.0
{'name': 'quux'}      quux        60.0   5.0
{'name': 'corge'}    corge        55.0  10.0
{'name': 'bar'}        bar        20.0   5.0
{'name': 'baz'}        baz         5.0   5.0
{'name': 'grault'}  grault        10.0  10.0
{'name': 'garply'}  garply        15.0  15.0
{'name': 'grault'}  grault        10.0  10.0
{'name': 'waldo'}    waldo        50.0   0.0
{'name': 'fred'}      fred        35.0   5.0
{'name': 'plugh'}    plugh         5.0   5.0
{'name': 'xyzzy'}    xyzzy        25.0   5.0
{'name': 'thud'}      thud        25.0   5.0
{'name': 'baz'}        baz         5.0   5.0
{'name': 'garply'}  garply        15.0  15.0
{'name': 'garply'}  garply        15.0  15.0
{'name': '

Upon load, the tree visualization will show all trees and will color them with a unified legend. Trees can be recolored by their individual minimum and maximum by toggling the "Legends: unified" button to "Legends: indiv."

In [10]:
%loadVisualization rt_path multiple_trees

<IPython.core.display.Javascript object>

In [11]:
# Execute this cell first (without using myQuery yet)
%fetchData anotherQuery

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [12]:
sgf = multi_gf.filter(anotherQuery, squash=True)
print("Nodes returned:", len(sgf.graph))
print(sgf.tree())

Nodes returned: 1
    __          __       __         __ 
   / /_  ____ _/ /______/ /_  ___  / /_
  / __ \/ __ `/ __/ ___/ __ \/ _ \/ __/
 / / / / /_/ / /_/ /__/ / / /  __/ /_  
/_/ /_/\__,_/\__/\___/_/ /_/\___/\__/  v1.2.0

0.000 kap

Legend (Metric: time)
█ 0.00 - 0.00
█ 0.00 - 0.00
█ 0.00 - 0.00
█ 0.00 - 0.00
█ 0.00 - 0.00
█ 0.00 - 0.00

name User code    ◀  Only in left graph    ▶  Only in right graph

